# INITIAL SETUP

### INTRO + EXPLAINATION

In [1]:
print("Welcome to Tristan Marcum's Blackjack simulator!")
print("The key feature of this code is to have each unique card simulated.")
print("This isn't simply generating a random card, but tracking both drawable and drawn cards.")
print("This means it's technically possible to 'count' cards (if you know how).")
print("")

Welcome to Tristan Marcum's Blackjack simulator!
The key feature of this code is to have each unique card simulated.
This isn't simply generating a random card, but tracking both drawable and drawn cards.
This means it's technically possible to 'count' cards (if you know how).



In [2]:
# Import dependencies
import random
import copy
import numpy as np

### CUSTOM FUNCTIONS

In [3]:
# Defining dictionary layout of decks
def new_deck():
    return {"suits": [],
            "values": [],
            "ids": []}

# Takes an input, and randomly selects an index from the original, to one by one assemble a shuffled deck
def randomized_shuffling(deck):
    # Create a copy of the original deck to work with
    bin_deck = copy.deepcopy(deck)

    # Create a new deck to store the shuffled cards
    master_deck = new_deck()

    # Randomize the deck
    while len(bin_deck["suits"]) > 0:
        # Pick a random index
        random_index = random.randint(0, len(bin_deck["suits"]) - 1)

        # Add the card at the random index to the master deck
        for key in ["suits", "values", "ids"]:
            master_deck[key].append(bin_deck[key].pop(random_index))

    return master_deck

# Takes an input, and shuffles is in an alternating pattern
def interlaced_shuffling(deck):
    # Initial Setup
    temp_deck = copy.deepcopy(deck)

    # Randomize the deck a random number of times
    shuffle_count = random.randint(10, 20)
    for _ in range(shuffle_count):
        # Create a temporary deck to store the shuffled cards
        master_deck = new_deck()

        # Interlace shuffle
        half = len(temp_deck["suits"]) // 2
        for card in range(half):
            for key in ["suits", "values", "ids"]:
                master_deck[key].append(temp_deck[key][half + card])
                master_deck[key].append(temp_deck[key][card])

    return master_deck

# Takes 2 inputs, and seperates them accross 52 bins in a random order before reassembling by bin
def rotary_shuffling(master_deck, discard_deck):
    # Port and merge to temp dictionary
    temp_deck = new_deck()
    for key in temp_deck:
        temp_deck[key].extend(discard_deck[key])
        temp_deck[key].extend(master_deck[key])

    # Resets
    master_deck = new_deck()
    discard_deck = new_deck()

    # Rotary shuffler reset / setup
    rotary_shuffler = {}
    slot_bins = list(range(52))
    open_bins = list(range(52))

    for id in range(52):
        # Building out dictionary with 52 bins
        slot_name = f"Bin {id}"
        rotary_shuffler[slot_name] = new_deck().copy()

    # Loop until all cards are binned
    while temp_deck["suits"]:
        # Randomized target
        current_slot = random.choice(open_bins)
        slot_name = f"Bin {current_slot}"

        # Checking if bin has room, and transferring values if able
        if len(rotary_shuffler[slot_name]["suits"]) <= 8:
            for key in ["suits", "values", "ids"]:
                rotary_shuffler[slot_name][key].append(temp_deck[key].pop(0))
            if len(rotary_shuffler[slot_name]["suits"]) == 8:
                open_bins.remove(current_slot)

    # Dump bins to new master_deck
    opposite_slot = random.choice(slot_bins) + 25

    while slot_bins:
        # Adjustments to stay within correct index range
        opposite_slot %= 52
        slot_name = f"Bin {opposite_slot}"

        # Transferring values back to master_deck
        for key in ["suits", "values", "ids"]:
            master_deck[key].extend(rotary_shuffler[slot_name][key])

        del rotary_shuffler[slot_name]
        slot_bins.remove(opposite_slot)

        # Move to next bin
        opposite_slot += 1

    # Return the shuffled master_deck and the emptied/reset discard_deck
    return master_deck, discard_deck



### INITIAL DECK BUILD

In [4]:
# Unicode icons
spade = '\u2660'
club = '\u2663'
heart = '\u2665'
diamond = '\u2666'

#setting up unique features
suits = [spade,diamond,club,heart]
faces = ["A","2","3","4","5","6","7","8","9","10","J","Q","K"]
values = [11,2,3,4,5,6,7,8,9,10,10,10,10]

# initial setup
raw_deck = {"suits": [],
            "values": []}

# loop to initalize all unique combinations in both bins
for suit_index,suit in enumerate(suits):
    # forward iteration for first 2 suits
    if suit_index < 2:
        for face_index, face in enumerate(faces):
            raw_deck["suits"] += [suit + face]
            raw_deck["values"] += [values[face_index]]
    # reverse iteration for last 2 suits
    else:
        for face_index, face in enumerate(reversed(faces)):
            raw_deck["suits"] += [suit + face]
            raw_deck["values"] += [values[-face_index-1]]
            
# Now building a deck of 8 decks in new deck order
raw_master = new_deck()

# loop to make master deck using 8 raw decks
for deck in range(8):
    raw_master["suits"] += raw_deck["suits"]
    raw_master["values"] += raw_deck["values"]
    
# loop to create unique ids for each of the cards made in the master deck
for id in range(len(raw_master["suits"])):
    raw_master["ids"] += [id]

In [5]:
# doing a random shuffle first
master_deck = randomized_shuffling(raw_master)
# then doing an interlaced shuffle to further randomize
master_deck = interlaced_shuffling(master_deck)

# initializing discard deck
discard_deck = new_deck()

# SIMULATION

### RESULT BIN

In [6]:
win_rates = {"PJDL":0, #Player blackjack, dealer lost
             "PJDB":0, #Player blackjack, dealer bust
             "PLDJ":0, #Player lost, dealer blackjack
             "PBDJ":0, #Player bust, dealer blackjack
             "PJDJ":0, #Push, each with blackjacks
             "PWDL":0, #Player wins, dealer lost
             "PWDB":0, #Player wins, dealer bust
             "PLDW":0, #Player loss, dealer wins
             "PBDW":0, #Player bust, dealer wins
             "PBDB":0, #Player bust, dealer bust
             "PpDp":0, #Standard Push
             "Errors":0} #Error Check

### GAME LOOP

In [7]:
print("This game features 2 modes: Basic with just the hand values, or Pro with card faces.")
valid_modes = ['B','P']
play_mode_input = input('Do you want use (B)asic or (P)ro Mode?  ')

# check for valid response
while play_mode_input not in valid_modes:
    print('Sorry, that was an invalid response. Please respond with a single letter.')
    play_mode_input = input('Do you want use (B)asic or (P)ro Mode?  ')
    
print(f"Thank you, you have chosen mode {play_mode}.")
print("")

This game features 2 modes: Basic with just the hand values, or Pro with card faces.
Do you want use (B)asic or (P)ro Mode?  B
Thank you, you have chosen B mode.



In [8]:
# randomly picks a shuffle trigger between the first and last 50 cards
cut_target = random.randint(49,(len(master_deck['suits']) - 51))

# Initial Setup
shuffles = 0
games = 0
dealer = new_deck()
player = new_deck()
shuffle_trigger = False
draw_live = True
game_live = True
valid_play = ['H','S']
valid_round = ['Y','N']
last hand = {}

# loop for X rounds
while game_live:
    #_____________________________________________________
    # Shuffle if trigger updated
    if shuffle_trigger:
        master_deck, discard_deck = rotary_shuffling(master_deck, discard_deck)
        shuffles += 1
        # generate new cut target
        cut_target = random.randint(49,(len(master_deck['suits']) - 51))
        print("Deck shuffled.")
    
    # Shuffle next round if cut target is reached
    if len(master_deck['suits']) <= cut_target:
        print("Cut-Card drawn. Deck to be shuffled next round.")
        shuffle_trigger = True
        
    #_____________________________________________________
    # Initial Draws
    
    for deal in range(2):
        for key in ["suits", "values", "ids"]:
            dealer[key].append(master_deck[key].pop(0))
        for key in ["suits", "values", "ids"]:
            player[key].append(master_deck[key].pop(0))
            
    #_____________________________________________________      
    # Instant round check
    if sum(dealer['values']) == 21 and sum(player['values']) == 21:
        print('Both you and the dealer were dealt blackjacks. This round is a draw.')
    elif sum(dealer['values']) == 21:
        print('Dealer was dealt a blackjack. Better luck next round.')
    
    # Standard game loop
    else:
        #Player's Turn
        draw_live = True #Round reset
        
        # Double Ace check
        if sum(player['values']) == 22:
            player['values'][0] = 1
            print("You were dealt 2 aces. One will be flipped to a value of 1.")
          
        # Player skip check
        if sum(player['values']) == 21:
                print('Lucky you! You were dealt a Blackjack!') #skip player input
        #_____________________________________________________
        #Simple mode
        elif play_mode == "B":
            
            #_____________________________________________________
            #Player's Turn
              
            # initial round prints
            print(f"Dealer is showing {dealer['values'][0]}")
            print(f"Your hand is {sum(player['values'])}. What would you like to do?")

            # loop until bust or stop
            while draw_live:
                # player resposne
                play_input = input('Do you want to (H)it or (S)tand?  ')

                # check for valid response
                while play_input not in valid_play:
                    print('Sorry, that was an invalid response. Please respond with a single letter.')
                    play_input = input('Do you want to (H)it or (S)tand?  ')

                # check response
                if play_input == "H":
                    for key in ["suits", "values", "ids"]:
                        player[key].append(master_deck[key].pop(0))
                    print(f"You drew a new card. Your hand is now {sum(player['values'])}")
                else:
                    draw_live = False
                    
                if play_input == "H":
                    for key in ["suits", "values", "ids"]:
                        player[key].append(master_deck[key].pop(0))
                    if sum(player['values']) > 21:
                        if 11 in player['values']:
                            player_ace = player['values'].index(11)
                            player['values'][player_ace] = 1
                            print("You drew an Ace, but you almost went over. Ace flipped to 1")
                            print(f"Your hand is now {sum(player['values'])}")
                    else:
                        print(f"You drew a new card. Your hand is now {sum(player['values'])}")
                else:
                    draw_live = False

                # bust check
                if sum(player['values']) > 21:
                    print('Sorry, you went bust. Better luck next round.')
                    draw_live = False

            #_____________________________________________________
            #Dealer's Turn
            if sum(dealer['values']) == 22:
                dealer_ace = 0 
                dealer['values'][dealer_ace] = 1 #Update dual ace deal before hand check
            if sum(dealer['values']) > 17:
                print(f"Dealer stood with {sum(dealer['values'])}")
            else:
                while sum(dealer['values']) < 17:
                    for key in ["suits", "values", "ids"]:
                        dealer[key].append(master_deck[key].pop(0))
                    if sum(dealer['values']) > 21:
                        if 11 in dealer['values']:
                            dealer_ace = dealer['values'].index(11)
                            dealer['values'][dealer_ace] = 1
                print(f"Dealer drew to {sum(dealer['values'])}")
                
        #_____________________________________________________
        #Pro mode
        else:
            #_____________________________________________________
                              
            # initial round prints
            print(f"Dealer is showing {dealer['suits'][0]}")
            print(f"Your hand is {player['suits']}. What would you like to do?")

            # loop until bust or stop
            while draw_live:
                # player resposne
                play_input = input('Do you want to (H)it or (S)tand?  ')

                # check for valid response
                while play_input not in valid_play:
                    print('Sorry, that was an invalid response. Please respond with a single letter.')
                    play_input = input('Do you want to (H)it or (S)tand?  ')

                # check response
                if play_input == "H":
                    for key in ["suits", "values", "ids"]:
                        player[key].append(master_deck[key].pop(0))
                    if sum(player['values']) > 21:
                        if 11 in player['values']:
                            player_ace = player['values'].index(11)
                            player['values'][player_ace] = 1
                            print("You drew an Ace, but you almost went over. Ace flipped to 1")
                            print(f"Your hand is now {player['suits']}")
                    else:
                        print(f"You drew a new card. Your hand is now {player['suits']}")
                else:
                    draw_live = False

                # bust check
                if sum(player['values']) > 21:
                    print('Sorry, you went bust. Better luck next round.')
                    draw_live = False

            #_____________________________________________________
            #Dealer's Turn
            if sum(dealer['values']) == 22:
                dealer['values'][0] = 1 #Update dual ace deal before hand check
            if sum(dealer['values']) > 17:
                print(f"Dealer stood with {sum(dealer['values'])}")
            else:
                while sum(dealer['values']) < 17:
                    for key in ["suits", "values", "ids"]:
                        dealer[key].append(master_deck[key].pop(0))
                    if sum(dealer['values']) > 21:
                        if 11 in dealer['values']:
                            dealer_ace = dealer['values'].index(11)
                            dealer['values'][dealer_ace] = 1
                print(f"Dealer drew to {sum(dealer['values'])}")
    #____________________________________________________
    #Round Results
    
    # Player blackjack
    if sum(player['values']) == 21 and len(player['values']) == 2:
        if sum(dealer['values']) == 21:
            if len(dealer['values']) == 2:
                win_rates["PJDJ"] += 1  # Push, each with blackjack
                #print('Both you and the dealer were dealt blackjacks. This round is a draw.') #Already printed
            else:
                win_rates["PpDp"] += 1  # Standard Push
                print(f"So close! This round is a draw. Both you and the dealer got {sum(dealer['values'])}.")
        elif sum(dealer['values']) > 21:
            win_rates["PJDB"] += 1  # Player blackjack, dealer bust
            print('Great job! You got a winning Blackjack, and the dealer busted!')
        elif sum(dealer['values']) < 21:
            win_rates["PJDL"] += 1  # Player blackjack, dealer lost
            print('Great job! You got a winning Blackjack!')
        else:
            win_rates["Errors"] += 1  # Error bin

    # Dealer blackjack        
    elif sum(dealer['values']) == 21 and len(dealer['values']) == 2:
        if sum(player['values']) != 21:
            win_rates["PLDJ"] += 1  # Player lost, dealer blackjack
            #print('Dealer was dealt a blackjack. Better luck next round.') #Alredy printed
        else:
            win_rates["Errors"] += 1  # Error bin

    # Player valid        
    elif sum(player['values']) <= 21: 
        if sum(dealer['values']) <= 21:
            if sum(player['values']) > sum(dealer['values']):
                win_rates["PWDL"] += 1  # Player wins, dealer lost
                print(f"Great job! You beat the dealer with {sum(player['values'])}!")
            elif sum(player['values']) < sum(dealer['values']):
                win_rates["PLDW"] += 1  # Player loss, dealer wins
                print(f"Better luck next round. The dealer managed to beat you with a {sum(dealer['values'])}.")
            elif sum(dealer['values']) > 21:
                win_rates["PWDB"] += 1  # Player wins, dealer bust (#1)
                print(f"Great job! The dealer bust, so your hand of {sum(player['values'])} wins!")
            elif sum(player['values']) == sum(dealer['values']):
                win_rates["PpDp"] += 1  # Standard Push
                print(f"So close! Both you and the dealer managed to get {sum(player['values'])}.")
            else:
                win_rates["Errors"] += 1  # Error bin
        else:
            win_rates["PWDB"] += 1  # Player wins, dealer bust (#2)
            print(f"Great job! The dealer bust, so you hand of {sum(player['values'])} wins.")
            
    # Player busts
    else:
        if sum(dealer['values']) <= 21:  # Dealer valid
            win_rates["PBDW"] += 1  # Player bust, dealer wins
            print(f"Better luck next round. You went bust, so the dealer's hand of {sum(dealer['values'])} won this round.")
        elif sum(dealer['values']) > 21:
            win_rates["PBDB"] += 1  # Player bust, dealer bust
            print(f"So close! Both you and the dealer went over 21.")
        else:
            win_rates["Errors"] += 1  # Error bin

    # Ace reset
    if 1 in player['values']:
        player['values'][player_ace] = 11
    if 1 in dealer['values']:
        dealer['values'][dealer_ace] = 11
        
    # Discard Transfer
    last hand = player
    for key, value in dealer.items():
        if key in discard_deck:
            discard_deck[key] += value

    for key, value in player.items():
        if key in discard_deck:
            discard_deck[key] += value

    # Bin update
    dealer = new_deck()
    player = new_deck()
    games += 1
    
    # Checking to continue game
    round_input = input('Do you want to continue playing: (Y)es or (N)o?  ')
    
    while round_input not in valid_round:
        print('Sorry, that was an invalid response. Please respond with a single letter.')
        round_input = input('Do you want to continue playing: (Y)es or (N)o?  ')
        
    if round_input == "N":
        game_live = False
    else:
        print("")
    
print("Thanks for playing! Here's your results:")

Dealer is showing 8
Your hand is 12. What would you like to do?
Do you want to (H)it or (S)tand?  H
You drew a new card. Your hand is now 22
Sorry, you went bust. Better luck next round.
Dealer drew to 18
Better luck next round. You went bust, so the dealer's hand of 18 won this round.
Do you want to continue playing: (Y)es or (N)o?  Y

Dealer is showing 10
Your hand is 18. What would you like to do?
Do you want to (H)it or (S)tand?  S
Dealer drew to 22
Great job! The dealer bust, so you hand of 18 wins.
Do you want to continue playing: (Y)es or (N)o?  Y

Dealer is showing 10
Your hand is 9. What would you like to do?
Do you want to (H)it or (S)tand?  H
You drew a new card. Your hand is now 12
Sorry, you went bust. Better luck next round.
Dealer stood with 18
Better luck next round. You went bust, so the dealer's hand of 18 won this round.
Do you want to continue playing: (Y)es or (N)o?  Y

Dealer is showing 10
Your hand is 11. What would you like to do?
Do you want to (H)it or (S)tand

KeyboardInterrupt: Interrupted by user

# RESULTS

In [ ]:
# Calculating sum of results
total_plays = 0
for key in win_rates:
    total_plays += win_rates[key]

print(f"Player blackjack, dealer lost = {win_rates['PJDL']}")
print(f"Player blackjack, dealer bust = {win_rates['PJDB']}")
print(f"Player lost, dealer blackjack = {win_rates['PLDJ']}")
print(f"Player bust, dealer blackjack = {win_rates['PBDJ']}")
print(f"Push, each with blackjacks = {win_rates['PJDJ']}")
print(f"Player wins, dealer lost = {win_rates['PWDL']}")
print(f"Player wins, dealer bust = {win_rates['PWDB']}")
print(f"Player loss, dealer wins = {win_rates['PLDW']}")
print(f"Player bust, dealer wins = {win_rates['PBDW']}")
print(f"Player bust, dealer bust = {win_rates['PBDB']}")
print(f"Standard Push = {win_rates['PpDp']}")
#print(f"Error Count = {win_rates['Errors']}")
print("") # space
win_rate = (win_rates['PJDL']+
            win_rates['PJDB']+
            win_rates['PWDL']+
            win_rates['PWDB'])
print(f"Win Rate = {(win_rate/total_plays)*100}%")

In [2]:
test = {'test1': 0,
       'test2': "test2",
       'test3': ["test3"]}

In [3]:
print(test['test2'])

test2


In [ ]:
---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
~\AppData\Local\Temp\ipykernel_16080\1181724005.py in <module>
     64         #_____________________________________________________
     65         # Manual play
---> 66         player_hands, dealer_hand = deal_loop_manual()
     67 
     68         # Print Results

~\AppData\Local\Temp\ipykernel_16080\3617947941.py in deal_loop_manual()
     68                     #_____________________________________________________
     69                     # input seperation
---> 70                     input_selection = hand_options[list(hand_option.keys())[0]]
     71 
     72                     if input_selection == 'H':

NameError: name 'hand_option' is not defined


In [3]:
testy = {'besty': [2,3,4]}

print(sum(testy['besty']))
print(len(testy['besty']))

9
3
